In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# --- Cargar los datos ---
datos = load_breast_cancer()
X_numpy = datos.data
Y_numpy = datos.target # 0 = benigno, 1 = maligno

# --- Dividir en Entrenamiento y Prueba (Train/Test Split) ---
# Es VITAL evaluar el modelo con datos que nunca ha visto.

X_train_np, X_test_np, Y_train_np, Y_test_np = train_test_split(
    X_numpy, Y_numpy, test_size=0.2, random_state=42
)

# --- Escalar los datos ---
# Las redes neuronales funcionan mejor cuando las características
# están en una escala similar (ej. media 0, desviación 1).
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_test_np = scaler.transform(X_test_np) # ¡Usar la misma escala de 'fit'!

# --- Convertir de NumPy a Tensores de PyTorch ---
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)

# La función de pérdida (BCELoss) espera que la Y tenga la misma forma que la salida del modelo.
# Si nuestro modelo saca (N, 1), Y debe ser (N, 1), no solo (N,).
Y_train = torch.tensor(Y_train_np, dtype=torch.float32).view(-1, 1)
Y_test = torch.tensor(Y_test_np, dtype=torch.float32).view(-1, 1)

# Obtener dimensiones
n_samples, n_features = X_train.shape

In [4]:
class Clasificador(nn.Module):
    def __init__(self, input_dim):
        super(Clasificador, self).__init__()
        
        # Definimos las capas de forma secuencial
        self.net = nn.Sequential(
            # Capa de entrada (30 características) a primera capa oculta (16 neuronas)
            nn.Linear(input_dim, 16),
            nn.ReLU(),  # Activación no lineal
            
            # Segunda capa oculta (16 neuronas) a tercera (8 neuronas)
            nn.Linear(16, 8),
            nn.ReLU(),
            
            # Capa de salida (8 neuronas) a la salida final (1 neurona)
            nn.Linear(8, 1),
            nn.Sigmoid() # Salida de probabilidad (0 a 1)
        )
    
    def forward(self, x):
        return self.net(x)

# Instanciamos el modelo
model = Clasificador(n_features)
print(model) # Imprime la arquitectura

Clasificador(
  (net): Sequential(
    (0): Linear(in_features=30, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [5]:
learning_rate = 0.001
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
n_epochs = 100

for epoch in range(n_epochs):
    # 1. Forward pass
    y_pred = model(X_train)
    
    # 2. Calcular Loss
    loss = loss_fn(y_pred, Y_train)
    
    # 3. Resetear gradientes
    optimizer.zero_grad()
    
    # 4. Backward pass (Autograd)
    loss.backward()
    
    # 5. Actualizar pesos
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.7058
Epoch [20/100], Loss: 0.6631
Epoch [30/100], Loss: 0.6115
Epoch [40/100], Loss: 0.5522
Epoch [50/100], Loss: 0.4867
Epoch [60/100], Loss: 0.4195
Epoch [70/100], Loss: 0.3569
Epoch [80/100], Loss: 0.3027
Epoch [90/100], Loss: 0.2580
Epoch [100/100], Loss: 0.2224


In [7]:
# Ponemos el modelo en "modo evaluación"
# Esto desactiva el cálculo de gradientes (más rápido)
# y desactiva capas como Dropout (si las hubiera).
with torch.no_grad():
    y_pred_test = model(X_test)
    
    # Convertimos las probabilidades (0 a 1) en clases (0 o 1)
    # Si la probabilidad es > 0.5, es 1 (maligno), si no, 0 (benigno)
    predichos = (y_pred_test > 0.5).float()
    
    # Calculamos la precisión (accuracy)
    # Comparamos cuántas predicciones (predichos) son iguales a la verdad (Y_test)
    accuracy = (predichos == Y_test).float().mean()
    
    print(f'\n--- Evaluación en el conjunto de Prueba ---')
    print(f'Precisión (Accuracy): {accuracy.item() * 100:.2f}%')


--- Evaluación en el conjunto de Prueba ---
Precisión (Accuracy): 97.37%
